# gdmongolite Beginner Tutorial

Welcome to the world's easiest MongoDB toolkit! This notebook will teach you everything you need to know to become a MongoDB pro.

## What you'll learn:
- Connect to MongoDB with zero configuration
- Create data models with automatic validation
- Perform CRUD operations (Create, Read, Update, Delete)
- Write complex queries with simple syntax
- Handle errors gracefully

## Step 1: Installation and Import

First, install gdmongolite if you haven't already:

In [ ]:
# Uncomment and run this if you need to install gdmongolite
# !pip install gdmongolite

In [ ]:
# Import everything you need in one line!
from gdmongolite import DB, Schema, Email, FieldTypes

print("gdmongolite imported successfully!")

## Step 2: Connect to Database

gdmongolite automatically connects to MongoDB with zero configuration:

In [ ]:
# Connect to database (automatic!)
db = DB()
print("Connected to MongoDB automatically!")

## Step 3: Define Your Data Model

Think of a Schema like defining columns in an Excel sheet, but much smarter:

In [ ]:
class User(Schema):
    """
    User model with automatic validation
    """
    name: FieldTypes.Name        # Validates names (1-100 chars)
    email: Email                 # Validates email format
    age: FieldTypes.Age         # Validates age (0-150)
    hobbies: list[str] = []     # List of hobbies (optional)
    is_active: bool = True      # Default to active

# Register the model with database
db.register_schema(User)
print("User model registered!")

## Step 4: Create Data (INSERT)

Let's save some users to the database:

In [ ]:
# Save one user
response = await db.User.insert({
    "name": "Alice Johnson",
    "email": "alice@example.com",
    "age": 28,
    "hobbies": ["reading", "coding", "hiking"]
})

print(f"Success: {response.success}")
print(f"Message: {response.message}")
print(f"User ID: {response.data}")

In [ ]:
# Save multiple users at once
users_data = [
    {
        "name": "Bob Smith",
        "email": "bob@example.com",
        "age": 35,
        "hobbies": ["gaming", "cooking"]
    },
    {
        "name": "Carol Davis",
        "email": "carol@example.com",
        "age": 42,
        "hobbies": ["painting", "yoga", "traveling"]
    },
    {
        "name": "David Wilson",
        "email": "david@example.com",
        "age": 29,
        "hobbies": ["music", "photography"]
    }
]

response = await db.User.insert(users_data)
print(f"Saved {response.count} users!")

## Step 5: Read Data (FIND)

Now let's find users with various queries:

In [ ]:
# Find all users
all_users = await db.User.find().to_list()
print(f"Total users: {len(all_users)}")

# Show first user
if all_users:
    print(f"First user: {all_users[0]['name']} ({all_users[0]['email']})")

In [ ]:
# Find users by age (30 or older)
adults = await db.User.find(age__gte=30).to_list()
print(f"Users 30+: {len(adults)}")

for user in adults:
    print(f"  - {user['name']}: {user['age']} years old")

In [ ]:
# Find users with specific hobbies
coders = await db.User.find(hobbies__contains="coding").to_list()
print(f"Users who code: {len(coders)}")

for user in coders:
    print(f"  - {user['name']}: {user['hobbies']}")

In [ ]:
# Find one specific user
alice = await db.User.find(name="Alice Johnson").first()
if alice:
    print(f"Found Alice!")
    print(f"Email: {alice['email']}")
    print(f"Age: {alice['age']}")
    print(f"Hobbies: {', '.join(alice['hobbies'])}")

## Step 6: Update Data (UPDATE)

Let's modify some user data:

In [ ]:
# Add a new hobby to Alice
response = await db.User.update(
    {"name": "Alice Johnson"},  # Find Alice
    {"$push": {"hobbies": "swimming"}}  # Add swimming to hobbies
)

print(f"Updated {response.count} users")

# Check Alice's updated hobbies
alice = await db.User.find(name="Alice Johnson").first()
print(f"Alice's hobbies now: {alice['hobbies']}")

In [ ]:
# Make everyone one year older
response = await db.User.update(
    {},  # Empty filter = all users
    {"$inc": {"age": 1}}  # Increment age by 1
)

print(f"Made {response.count} users one year older!")

# Check updated ages
users = await db.User.find().to_list()
for user in users:
    print(f"  - {user['name']}: {user['age']} years old")

## Step 7: Advanced Queries

Let's try some more complex queries:

In [ ]:
# Complex query: users between 25-40 who like coding
tech_users = await db.User.find(
    age__gte=25,
    age__lte=40,
    hobbies__contains="coding"
).to_list()

print(f"Tech users (25-40): {len(tech_users)}")
for user in tech_users:
    print(f"  - {user['name']}: {user['age']} years, hobbies: {user['hobbies']}")

In [ ]:
# Sorting: get users by age (youngest first)
sorted_users = await db.User.find().sort("age").to_list()

print("Users by age (youngest first):")
for user in sorted_users:
    print(f"  - {user['name']}: {user['age']} years old")

In [ ]:
# Projection: get only names and emails
names_emails = await db.User.find().project("name", "email").to_list()

print("Names and emails only:")
for user in names_emails:
    print(f"  - {user['name']}: {user['email']}")

In [ ]:
# Pagination: get first 2 users
first_page = await db.User.find().limit(2).to_list()
print(f"First page: {len(first_page)} users")

# Get next 2 users
second_page = await db.User.find().skip(2).limit(2).to_list()
print(f"Second page: {len(second_page)} users")

## Step 8: Error Handling

See how gdmongolite handles invalid data:

In [ ]:
# Try to save invalid data
response = await db.User.insert({
    "name": "",  # Empty name (invalid!)
    "email": "not-an-email",  # Invalid email!
    "age": -5  # Negative age (invalid!)
})

print(f"Success: {response.success}")
if not response.success:
    print(f"Error caught: {response.error}")
    print("gdmongolite protected your database from bad data!")

## Step 9: Delete Data (DELETE)

Finally, let's clean up some data:

In [ ]:
# Delete users over 50 (none in our example)
response = await db.User.delete(age__gt=50)
print(f"Deleted {response.count} users over 50")

# Count remaining users
remaining = await db.User.find().count()
print(f"Users remaining: {remaining}")

## Step 10: Synchronous Operations

If you prefer non-async code, gdmongolite supports that too:

In [ ]:
# Sync operations (no async/await needed)

# Save user (sync)
response = db.User.insert_sync({
    "name": "Sync User",
    "email": "sync@example.com",
    "age": 25
})
print(f"Sync save: {response.message}")

# Find users (sync)
users = db.User.find(age__gte=20).to_list_sync()
print(f"Found {len(users)} users (sync)")

# Update user (sync)
response = db.User.update_sync(
    {"name": "Sync User"},
    {"age": 26}
)
print(f"Sync update: {response.message}")

# Delete user (sync)
response = db.User.delete_sync(name="Sync User")
print(f"Sync delete: {response.message}")

## Congratulations!

You've just learned MongoDB with gdmongolite! You now know how to:

- Connect to MongoDB with zero configuration
- Define data models with automatic validation
- Save data to the database
- Find data with simple and complex queries
- Update data safely
- Delete data when needed
- Handle errors gracefully
- Use both async and sync operations

## Next Steps:

1. Try the Web API notebook to create REST APIs
2. Try the Data Import/Export notebook to work with files
3. Explore the advanced features in the documentation
4. Build something amazing!

You're now a MongoDB pro with gdmongolite!